In [ ]:
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp

## Flip Coin Pipeline 

In [ ]:
# Define a class extensing ContainerOp
class FlipCoinOp(dsl.ContainerOp):

    def __init__(self, name):
        super(FlipCoinOp, self).__init__(
            name=name,
            image='python:alpine3.6',
            command=['sh', '-c'],
            arguments=['python -c "import random; result = \'heads\' if random.randint(0,1) == 0 '
                       'else \'tails\'; print(result)" | tee /tmp/output'],
            file_outputs={'output': '/tmp/output'})

class PrintOp(dsl.ContainerOp):

    def __init__(self, name):
        super(PrintOp, self).__init__(
          name=name,
          image='alpine:3.6',
          command=['echo', '"it was tail"'])

In [ ]:
# Define a pipeline
@dsl.pipeline(
    name='pipeline flip coin loop',
    description='shows how to use dsl.Loop.'
)

def flipcoinloop():
    flip = FlipCoinOp('flip')

    with dsl.Condition(flip.output=='tails'):
        PrintOp('tails')
        flip = FlipCoinOp('flip2')

    with dsl.Condition(flip.output=='heads'):
        PrintOp('heads')

In [ ]:
compiler.Compiler().compile(flipcoinloop, 'flip_coin_2.tar.gz')

In [ ]:
client = kfp.Client()

In [ ]:
client.list_experiments()

In [ ]:
client.run_pipeline(experiment_id='',
                    job_name='second flip run',
                    pipeline_package_path='flip_coin_2.tar.gz')